# Source extraction with manually selected cells

This is the pipeline for manual source extraction from the motion-corrected memmap file.

CaImAn offers the option to substitute the automatic search for cells by a mask of manually selected neurons. This can be useful in cases where CaImAns automatic extraction does not work or simply if you want to do the manual work.

To manually select ROIs, we can use Adrians GUI. It loads the local correlation and mean intensity images of a session where we can select coordinates of ROIs. These are exported as a `.txt` file. Different from the automatic extraction, these neuron coordinates are used to create spatial masks with a circular kernel around it (radius = `gSig[0]`). These masks are then fed into the CNMF object which refines the ROI borders and extracts the calcium signal from it.

Because manually selected cells are assumed to already be actual cells, we are not performing component evaluation here, but directly detrending the data and saving the finished cnm object.

In [1]:
# Import packages
import sys
sys.path.append('../custom scripts/')

from caiman.source_extraction import cnmf
import place_cell_pipeline as pipe
import behavior_import as behavior
import caiman as cm
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import matplotlib.gridspec as gridspec
from datetime import datetime
import preprocess as pre

# Start cluster for parallel processing
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)

### Define parameters

First, we have to define the root directory and source extraction parameters.

In [2]:
root = r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M37\20200323'

# dataset dependent parameters
fr = 30                       # imaging rate in frames per second
decay_time = 0.4              # length of a typical transient in seconds (0.4)
dxy = (0.83, 0.76)            # spatial resolution in x and y in (um per pixel) [(1.66, 1.52) for 1x, (0.83, 0.76) for 2x]


# extraction parameters
p = 1                         # order of the autoregressive system
gnb = 2                       # number of global background components (3)
merge_thr = 0.75              # merging threshold, max correlation allowed (0.86)
rf = 25                       # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 10              # amount of overlap between the patches in pixels (20)
K = 12                        # number of components per patch (10)
gSig = [12, 12]               # expected half-size of neurons in pixels [X, Y] (has to be int, not float!)                 
method_init = 'greedy_roi'    # initialization method (if analyzing dendritic data using 'sparse_nmf')
ssub = 2                      # spatial subsampling during initialization
tsub = 2                      # temporal subsampling during intialization

opts_dict = {'fnames': None,
             'nb': gnb,
             'rf': rf,
             'K': K,
             'gSig': gSig,
             'stride': stride_cnmf,
             'method_init': method_init,
             'rolling_sum': True,
             'merge_thr': merge_thr,
             'only_init': True,
             'ssub': ssub,
             'tsub': tsub}

cnm_params = cnmf.params.CNMFParams(params_dict=opts_dict)


## Manually select ROIs with Adrians GUI

Adrians GUI is useful to set coordinates for neuron-ROIs. It is run by calling the large cell below and loads the images for the session defined by `root`.

In [ ]:
%matplotlib qt

cor = io.imread(root + r'\local_correlation_image.tif')
avg = io.imread(root + r'\mean_intensity_image.tif')

###  Plot average and correlation image
fig = plt.figure()
plt.clf()
curr_sess = datetime.strptime(os.path.split(root)[1], '%Y%m%d').strftime('%d.%m.%Y')
curr_mouse = os.path.split(os.path.split(root)[0])[1]
fig.canvas.set_window_title(f'SELECT NEURONS  Mouse: {curr_mouse}  Day: {curr_sess}')

gs = gridspec.GridSpec(2, 2, height_ratios=[10, 1])
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[0, 1], sharex=ax0, sharey=ax0)
txt = plt.subplot(gs[1, 0:2])

# text_box = TextBox(txt, 'Status:', initial='Click to select neurons, backspace to remove last neuron, enter when you are done!')
txt.text(x=0.3, y=0.1, s='Click to select neurons, backspace to remove last neuron, enter when you are done!')
txt.get_xaxis().set_visible(False)
txt.get_yaxis().set_visible(False)
txt.get_xaxis().set_ticks([])
txt.get_yaxis().set_ticks([])

# Average image
minA, maxA = np.percentile(avg, [5, 99.9])
im0 = ax0.imshow(avg, vmin=minA, vmax=maxA)
ax0.set_title('Average image')

# Correlation image
minC, maxC = np.percentile(cor, [5, 99.9])
im1 = ax1.imshow(cor, vmin=minC, vmax=maxC)
ax1.set_title('Correlation image')

# set adjustable colorbar

cbar0 = fig.colorbar(im0, ax=ax0, fraction=0.046, pad=0.04)
cbar0 = pre.DraggableColorbar(cbar0, ax0, im0, vmin=minA, vmax=maxA)
cbar0.connect()

cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
cbar1 = pre.DraggableColorbar(cbar1, ax1, im1, vmin=minC, vmax=maxC)
cbar1.connect()


def drawCross(ax, x=20, y=20):
    ax.axvline(x=x, color='w', LineWidth = 0.5)
    ax.axhline(y=y, color='w', LineWidth = 0.5)

crosses = list()
crosses1 = list()
pos_x = list()
pos_y = list()

def func(event, ax, ax1):
    x = int(event.xdata)
    y = int(event.ydata)

    # print('Click location:\tX: ',x , '\tY: ', y)

    # draw a cross at the click location of axis (also ax1 if not None)
    cross = ax.plot(x, y, 'x', color='red')
    cross1 = ax1.plot(x, y, 'x', color='red')

    crosses.append(cross)
    crosses1.append(cross1)
    pos_x.append(x)
    pos_y.append(y)

    ax.figure.canvas.draw()


# connect click events to avg and cor axes
click = pre.Click(axes=ax0, func=func, second_axes=ax1, button=1)


### Read keyboard enter and backspace events
def press(event):
    if event.key == 'backspace':
        print('Removing last element')

        # remove last element from list
        pos_x.pop()
        pos_y.pop()
        # remove also from plot and refresh plot
        crosses.pop()[0].remove()
        crosses1.pop()[0].remove()
        ax0.figure.canvas.draw()

    if event.key == 'enter':
        ### Save the clicked positions to file
        file_name = os.path.join(root, 'clicked_neurons_{}.txt'.format(datetime.now().strftime('%Y%m%d_%H%M%S')))

        with open(file_name, 'w') as file:
            # write header
            file.write('Neuron\tX\tY\n')
            # write neuron positions in each line
            i = 0
            for x, y in zip(pos_x, pos_y):
                file.write('{}\t{}\t{}\n'.format(i, x, y))
                i += 1

        print('Wrote positions of {} neurons to file {}'.format(i, file_name))
        txt.clear()
        txt.text(x=0.3, y=0.1, s='Wrote positions of {} neurons to file {}'.format(i, file_name))
        # text_box = TextBox(txt, 'Status:', initial='Wrote positions of {} neurons to file {}'.format(i,file_name))
        ax0.figure.canvas.draw()


# connect keyboard events
fig.canvas.mpl_connect('key_press_event', press)

# show the created figure and wait till figure is closed
# plt.tight_layout()
plt.show()



## Source extraction

Now we can perform the real source extraction. For this we...<br>
1. ...load the motion-corrected memmap file,<br>
2. ...load the clicked_neuron.txt file that contains manually selected ROIs,<br>
3. ...run CaImAns source extraction function on these ROIs,<br>
4. ...plot the contours of all components.<br>

This all happens under the hood of the `manual_neuron_extraction()` function.

Note that if you did several different selection of ROIs in one session, you can specify which .txt file to load by setting the `fname` parameter.

In [4]:
# Load memmap file
mmap_file, images = pipe.load_mmap(root)
cnm_params = cnm_params.change_params({'fnames': mmap_file})

# Run source extraction with manually selected ROIs
cnm = pipe.manual_neuron_extraction(root, images, cnm_params, dview, fname=None)
pipe.save_cnmf(cnm, path=os.path.join(root, 'cnm_results_manual.hdf5'), verbose=False, overwrite=False)

# Load local correlation image (should have been created during motion correction)
cnm.estimates.Cn = io.imread(root + r'\local_correlation_image.tif')  
pipe.save_cnmf(cnm, path=os.path.join(root, 'cnm_results_manual.hdf5'), verbose=False, overwrite=True)

# Plot contours of all components
%matplotlib qt
cnm.estimates.plot_contours(img=cnm.estimates.Cn, display_numbers=True)
plt.tight_layout()
fig = plt.gcf()
fig.set_size_inches((10, 10))
plt.savefig(os.path.join(root, 'components_manual.png'))

Loading file W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch3\M37\20200323\memmap__d1_512_d2_472_d3_1_order_C_frames_21453_.mmap...
Loading .txt file: clicked_neurons_20200408_100816.txt!
spatial support for each components given by the user


C:\Anaconda3\envs\caiman\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


### Detrending

Now that we have our refined ROIs and the corresponding calcium traces, we can skip the evaluation step as mentioned above and directly detrend the data to get the dF/F traces for the neurons.

We can also apply Peter's spike prediction algorithm on the dataset to get a summable estimate of spikes in the signal. These can be saved in `cnm.estimates.spikes`.

In [5]:
cnm.params.data['dff_window'] = int(cnm.estimates.C.shape[1]/4)
cnm.estimates.detrend_df_f(quantileMin=8, frames_window=cnm.params.data['dff_window'])
cnm.estimates.spikes = 
# Save complete CNMF results
pipe.save_cnmf(cnm, path=os.path.join(root, 'cnm_results_manual.hdf5'), overwrite=True, verbose=False)

C:\Anaconda3\envs\caiman\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [ ]:
%matplotlib qt
# Plot contours of all components
cnm.estimates.plot_contours(img=cnm.estimates.Cn, display_numbers=True)
plt.tight_layout()

In [ ]:
pipe.save_cnmf(cnm, path=os.path.join(root, 'cnm_results_manual_2.hdf5'), overwrite=True, verbose=False)

In [ ]:
plt.figure()
plt.plot(cnm.estimates.C[120])

In [ ]:
cnm.params.get('temporal', 'p')

In [ ]:

# Export memmap
file = 'motionCorrected_session.tif'

path = os.path.join(root, file)

# load memory mapped file and transform it to 16bit and C order
import caiman as cm

corrected = cm.load(mmap_file)    # frames x height x width

corrected_int = np.array(corrected[:-1,:,:] , dtype='int16' )
corrected = None   # save memory

toSave_cOrder = corrected_int.copy(order='C')
corrected_int = None   # save memory

# if this throws an error, the tifffile version might be too old
# print('Tifffile version: {}'.format(tif.__version__) )
# upgrade it with: !pip install --upgrade tifffile
import tifffile as tif
tif.imwrite( path, data=toSave_cOrder)